<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonTasks/blob/master/PS_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mean-Variance Frontier with 2 risky assets

In [760]:
!pip install yfinance

In [761]:
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from math import *
import plotly.graph_objs as go

In [762]:
yf.pdr_override()

appleDataW = pdr.get_data_yahoo("AAPL", start="2010-01-01", end="2021-12-31", interval = "1wk")
pgDataW = pdr.get_data_yahoo("PG", start="2010-01-01", end="2021-12-31", interval = "1wk")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [763]:
aapWeeklylReturns = appleDataW['Adj Close'].pct_change()[1 :]
pgWeeklyReturns = pgDataW['Adj Close'].pct_change()[1 :]

frame = {'AAPL': aapWeeklylReturns,
         'PG': pgWeeklyReturns}

weeklyReturns = pd.DataFrame(frame)

In [764]:
weeklyReturns = weeklyReturns.dropna()
weeklyReturns


,AAPL,PG
Date,,
2010-01-08,-0.005461,0.017515
2010-01-15,-0.006494,-0.028256
2010-01-22,-0.042197,0.038150
2010-01-29,-0.036329,-0.004540
2010-02-05,0.034470,0.009121
...,...,...
2021-11-26,0.011239,-0.009418
2021-12-03,0.065950,0.041152
2021-12-10,-0.013176,0.050809


In [765]:
brs = 0.015

In [766]:
mus = (weeklyReturns.mean())
cov = weeklyReturns.cov()

Ожидаемая доходность:

In [767]:
mus

AAPL    0.005989
PG      0.002396
dtype: float64

Матрица ковариций, элементы на главной диагонали - дисперсия

In [768]:
cov

,AAPL,PG
AAPL,0.001329,0.000199
PG,0.000199,0.000454


In [769]:
#print('Ожидаемая доходность ' , '(AAPL) : ', cov['AAPL']['AAPL'])
#print('Ожидаемая доходность ' , '(PG) : ', cov['PG']['PG'])

Построим границу среднего стандартного отклонения для пары активов:

In [770]:
#- How many assests to include in each portfolio
n_assets = 2
#-- How many portfolios to generate
n_portfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

min_portfolio_variance = 1000
min_weights = []
min_asset = []

np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(weeklyReturns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
   
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    if(portfolio_E_Variance < min_portfolio_variance):
      min_portfolio_variance = portfolio_E_Variance
      min_weights = weights
      min_asset = assets

In [771]:
#-- Plot the risk vs. return of randomly generated portfolios
#- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0 #-- Include risk free rate here for sharpe ratio

#-- Create Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, 
                         y=mean_variance_pairs[:,0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [772]:
print(min_asset[0], ' : ', min_weights[0])
print(min_asset[1], ' : ', min_weights[1])

AAPL  :  0.18361998981543495
PG  :  0.8163800101845651


In [773]:
portfolio_min_variance_return = min_weights[0]*mus['PG'] + min_weights[1]*mus['AAPL']
portfolio_min_variance_return_std = sqrt(min_portfolio_variance)
print('Ожидаемая доходность : ', portfolio_min_variance_return)
print('Cтандартное отклонение : ', portfolio_min_variance_return_std)
temp = weeklyReturns['AAPL']*(min_weights[1]) + min_weights[0]*weeklyReturns['PG'] - brs

sharp = (temp.mean())/portfolio_min_variance_return_std
print('Коэффициент Шарпа : ', sharp)

Ожидаемая доходность :  0.005329305195947049
Cтандартное отклонение :  0.020177072540570575
Коэффициент Шарпа :  -0.4792912740244072


In [774]:
weeklyReturns["NR"] = np.full(weeklyReturns.shape[0], brs)
weeklyReturns

,AAPL,PG,NR
Date,,,
2010-01-08,-0.005461,0.017515,0.015
2010-01-15,-0.006494,-0.028256,0.015
2010-01-22,-0.042197,0.038150,0.015
2010-01-29,-0.036329,-0.004540,0.015
2010-02-05,0.034470,0.009121,0.015
...,...,...,...
2021-11-26,0.011239,-0.009418,0.015
2021-12-03,0.065950,0.041152,0.015
2021-12-10,-0.013176,0.050809,0.015


In [775]:
mus = (weeklyReturns.mean())
cov = weeklyReturns.cov()

In [776]:
#- How many assests to include in each portfolio
n_assets = 3
#-- How many portfolios to generate
n_portfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

min_portfolio_variance = 1000
min_weights = []
min_asset = []

np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(weeklyReturns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
   
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    if(portfolio_E_Variance < min_portfolio_variance):
      min_portfolio_variance = portfolio_E_Variance
      min_weights = weights
      min_asset = assets

In [777]:
#-- Plot the risk vs. return of randomly generated portfolios
#- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0 #-- Include risk free rate here for sharpe ratio

#-- Create Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, 
                         y=mean_variance_pairs[:,0], 
                      #- Add color scale for sharpe ratio   
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
#- Add title/labels
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [778]:
print(min_portfolio_variance, min_weights, min_asset)

2.22731927896188e-06 [0.01017965 0.92629719 0.06352317] ['AAPL' 'NR' 'PG']


In [779]:
import math

Определим минимальную доходность, достаточную для выполнения плана

In [780]:
min_ret = math.exp(math.log(1.5)/52) - 1

In [781]:
#- How many assests to include in each portfolio
n_assets = 3
#-- How many portfolios to generate
n_portfolios = 1000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

min_portfolio_variance = 1000
min_weights = []
min_asset = []
max_ret = 0

np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(weeklyReturns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
   
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    if(portfolio_E_Variance < min_portfolio_variance) and (portfolio_E_Return >= min_ret):
      max_ret = portfolio_E_Return
      min_portfolio_variance = portfolio_E_Variance
      min_weights = weights
      min_asset = assets

Оптимальный вариант:

In [782]:
print(min_asset[0], ' : ', min_weights[0])
print(min_asset[1], ' : ', min_weights[1])
print(min_asset[2], ' : ', min_weights[2])

AAPL  :  0.010179646121965633
NR  :  0.9262971855984979
PG  :  0.06352316827953641


In [783]:
portfolio_min_variance_return = min_weights[0]*mus[min_asset[0]] + min_weights[1]*mus[min_asset[1]]+ min_weights[2]*mus[min_asset[2]]
portfolio_min_variance_return_std = sqrt(min_portfolio_variance)


In [784]:
temp = weeklyReturns[min_asset[0]] * min_weights[0] + weeklyReturns[min_asset[1]] * min_weights[1] + weeklyReturns[min_asset[2]] * min_weights[2]

vol = np.std(temp)*sqrt(365/7)

temp = temp - brs
sharp = (temp.mean())/portfolio_min_variance_return_std
print("Ожидаемая доходность : ", portfolio_min_variance_return)
print("Волатильность : ", vol)
print("Коэффициент Шарпа : ", sharp)

Ожидаемая доходность :  0.014107625813361934
Волатильность :  0.010768146029325654
Коэффициент Шарпа :  -0.5979374583118502
